#### This notebook gets all input data: SPY, TLT, TAIL, XLK, VIX, and generates one dataframe with daily granularity and one dataframe with weekly granularity. These dataframes are saved individually into csv files. 

In [29]:
import datetime
import numpy as np
import pandas as pd

from Modules.Stock import Stock

In [30]:
# get all ETFs from data/eikon/US_ETFs (eikon database)

# get a list of Stocks (object Stock) from the folder containing all ETFs -> p_exchange='US_ETFs'
p_folderName='US_ETFs'    # name of folder where the database is kept 
v_etfsList = Stock.getStocksFromFiles(p_database='eikon', p_number=None, p_exchange=p_folderName, p_shuffle=False)

# select which ETFs from the ETF universe will be part of the analysis 
v_ricList = ['SPY', 'TLT.O', 'TAIL.K', 'XLK']
v_selEtfsList = [a for a in v_etfsList if a.id in v_ricList]

# merge all selected ETFs into a dataframe using the dates as index; the columns are the close prices 
df_prices = Stock.getAssetsAggregatedDf(p_assetsList = v_selEtfsList, p_column = 'close')

df_prices

XLK     SPY   TLT.O   TAIL.K
2002-01-02   24.60  115.53     NaN      NaN
2002-01-03   25.68  116.84     NaN      NaN
2002-01-04   25.84  117.62     NaN      NaN
2002-01-07   25.56  116.79     NaN      NaN
2002-01-08   25.75  116.52     NaN      NaN
...            ...     ...     ...      ...
2021-09-24  156.71  443.91  146.91  18.4900
2021-09-27  155.19  442.64  146.37  18.4900
2021-09-28  150.59  433.72  144.09  18.6500
2021-09-29  150.40  434.45  144.34  18.6301
2021-09-30  149.32  429.14  144.32  18.7600

[4972 rows x 4 columns]

In [31]:
# get VIX - US volatility index from yahoo/US_indexes/ (yahoo finance database)

#Daily
v_vix = pd.read_csv('data/yahoo/US_indexes/^VIX.csv', parse_dates=True)
v_vix['Date'] = pd.to_datetime(v_vix['Date'], infer_datetime_format=True)
v_vix.index = v_vix['Date'].values
v_vix = v_vix.loc['2002-01-02':'2021-09-30']
v_vix = v_vix.drop(['Date', 'Close', 'High', 'Low', 'Open', 'Volume'], axis=1)
v_vix.rename(columns = {'Adj Close': 'VIX'}, inplace = True)

v_vix

VIX
2002-01-02  22.709999
2002-01-03  21.340000
2002-01-04  20.450001
2002-01-07  21.940001
2002-01-08  21.830000
...               ...
2021-09-24  17.750000
2021-09-27  18.760000
2021-09-28  23.250000
2021-09-29  22.559999
2021-09-30  23.139999

[4972 rows x 1 columns]

In [32]:
# dataframe of daily prices from 02-01-2002 up to 30-09-2021
df_prices_daily = pd.concat([df_prices,v_vix], axis=1)
df_prices_daily.info()
df_prices_daily.tail(20)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4972 entries, 2002-01-02 to 2021-09-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   XLK     4972 non-null   float64
 1   SPY     4972 non-null   float64
 2   TLT.O   4830 non-null   float64
 3   TAIL.K  1130 non-null   float64
 4   VIX     4972 non-null   float64
dtypes: float64(5)
memory usage: 233.1 KB


XLK     SPY   TLT.O   TAIL.K        VIX
2021-09-02  158.73  453.19  149.54  18.7500  16.410000
2021-09-03  159.36  453.08  148.18  18.6700  16.410000
2021-09-07  159.40  451.46  146.93  18.6200  18.139999
2021-09-08  158.79  450.91  147.93  18.6900  17.959999
2021-09-09  158.19  448.98  149.72  18.8000  18.799999
2021-09-10  156.63  445.44  148.40  18.8100  20.950001
2021-09-13  156.68  446.58  149.30  18.7900  19.370001
2021-09-14  156.48  444.17  151.11  18.8900  19.459999
2021-09-15  157.85  447.88  150.58  18.7500  18.180000
2021-09-16  157.94  447.17  149.89  18.7500  18.690001
2021-09-17  155.48  441.40  149.17  18.7750  20.809999
2021-09-20  152.38  434.04  151.02  19.0200  25.709999
2021-09-21  152.44  433.63  150.89  18.9700  24.360001
2021-09-22  154.53  437.86  151.79  18.8500  20.870001
2021-09-23  156.60  443.18  148.36  18.6250  18.629999
2021-09-24  156.71  443.91  146.91  18.4900  17.750000
2021-09-27  155.19  442.64  146.37  18.4900  18.760000
2021-09-28  150.59  433.72  144.09  18.6500  23.250000
2021-09-29  150.40  434.45  144.34  18.6301  22.559999
2021-09-30  149.32  429.14  144.32  18.7600  23.139999

In [33]:
# dataframe of weekly prices from 02-01-2002 up to 30-09-2021 
# the index of the week correspond to "friday"
df_prices_weekly= df_prices_daily.resample('W', label='right').mean()
df_prices_weekly.index = df_prices_weekly.index - pd.DateOffset(days=2)
df_prices_weekly.info()
df_prices_weekly.tail(10)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1031 entries, 2002-01-04 to 2021-10-01
Freq: W-FRI
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   XLK     1031 non-null   float64
 1   SPY     1031 non-null   float64
 2   TLT.O   1002 non-null   float64
 3   TAIL.K  235 non-null    float64
 4   VIX     1031 non-null   float64
dtypes: float64(5)
memory usage: 48.3 KB


XLK       SPY    TLT.O     TAIL.K        VIX
2021-07-30  153.4220  439.6040  149.138  18.968000  18.238000
2021-08-06  154.1340  440.3940  150.110  18.984000  17.780000
2021-08-13  154.0040  443.9240  147.018  18.752000  16.144000
2021-08-20  154.3720  442.6820  149.622  18.924000  19.166000
2021-08-27  157.4580  448.1300  149.136  18.762000  17.278000
2021-09-03  159.1100  452.3720  149.058  18.712000  16.320000
2021-09-10  158.2525  449.1975  148.245  18.730000  18.962499
2021-09-17  156.8860  445.4400  150.010  18.791000  19.302000
2021-09-24  154.5320  438.5240  149.794  18.791000  21.464000
2021-10-01  151.3750  434.9875  144.780  18.632525  21.927499

In [35]:
# save the .csv files
df_prices_daily.to_csv(r'raw_daily_prices.csv', index = True)
df_prices_weekly.to_csv(r'raw_weekly_prices.csv', index = True)